In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from collections import Counter

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize

In [5]:
from scipy import stats
from scipy.stats import norm, gamma, linregress, skew
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering 
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, confusion_matrix, silhouette_score, silhouette_samples

In [6]:
def dump_pickle(file, filename):
    outfile = open(filename, 'wb')
    pickle.dump(file, outfile)
    outfile.close()

def load_pickle(filename):
    infile = open(filename, 'rb')
    file = pickle.load(infile)
    infile.close()
    return file

In [7]:
sns.set()

## Team

Read NBA cluster data

In [8]:
df = pd.read_csv('nba_cleaned.csv')
df.shape

(1774, 116)

In [12]:
cluster_dict = {2: 'Ballhandler (Iso+PnR)', 1: 'Ballhandler (PnR)',
                6: 'Big (Post Up+Roll)', 5: 'Big (Roll+Cut)',
                0: 'Wing (Off Screen)', 4: 'Wing (Hand Off)', 3: 'Wing (Spot Up+Transition)'}

In [28]:
def display_team_distribution(team, season=2021, 
                              df=df, cluster_dict=cluster_dict):
    print(f'{season}-{str(season+1)[2:]} {team}')
    output = df[(df['team'] == team) & (df['season'] == season)].copy()
    cols = ['player', 'mp', 
            'usg_pct', 'ts_pct', 'Overall_PPP', 'obpm', 'dbpm', 'bpm',
            'player_cluster', 'player_cluster_discription']
    output['player_cluster_discription'] = output['player_cluster'].map(cluster_dict)
    return output[cols].sort_values('mp', ascending=False).reset_index().drop('index', axis=1)

### Orlando Magic (#1)

- Point Diff: -8.5 (27th)
- Offense: 104.7 (30th)
- Defense: 113.1 (17th)

In [29]:
display_team_distribution('Orlando Magic')

2021-22 Orlando Magic


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Franz Wagner,2429,21.2,0.559,1.005,-0.4,-0.3,-0.7,1,Ballhandler (PnR)
1,Cole Anthony,2059,25.1,0.519,0.895,-0.6,-0.6,-1.2,1,Ballhandler (PnR)
2,Wendell Carter Jr.,1852,20.9,0.601,1.041,1.0,0.6,1.6,3,Wing (Spot Up+Transition)
3,Mo Bamba,1824,17.2,0.582,1.040,-0.2,0.9,0.7,3,Wing (Spot Up+Transition)
4,Chuma Okeke,1749,16.4,0.501,0.914,-1.6,1.5,-0.1,3,Wing (Spot Up+Transition)
5,Gary Harris,1730,16.4,0.571,1.038,-2.0,-0.4,-2.4,4,Wing (Hand Off)
6,Terrence Ross,1448,20.5,0.515,0.929,-2.5,-1.1,-3.7,4,Wing (Hand Off)
7,R.J. Hampton,1402,18.4,0.481,0.820,-4.0,-0.1,-4.1,3,Wing (Spot Up+Transition)
8,Jalen Suggs,1307,25.3,0.455,0.738,-5.3,-0.2,-5.6,1,Ballhandler (PnR)
9,Moritz Wagner,960,22.9,0.623,1.108,1.5,0.1,1.6,3,Wing (Spot Up+Transition)


### Oklahoma City Thunder (#2, 12)

- Point Diff: -8.6 (28th)
- Offense: 104.8 (29th)
- Defense: 113.4 (19th)

In [30]:
display_team_distribution('Oklahoma City Thunder')

2021-22 Oklahoma City Thunder


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Shai Gilgeous-Alexander,1942,30.8,0.557,0.992,2.9,0.5,3.4,2,Ballhandler (Iso+PnR)
1,Darius Bazley,1924,18.3,0.515,0.915,-2.1,0.3,-1.8,3,Wing (Spot Up+Transition)
2,Josh Giddey,1700,22.2,0.478,0.768,-1.2,0.3,-1.0,1,Ballhandler (PnR)
3,Luguentz Dort,1665,23.3,0.541,0.974,-1.5,-0.7,-2.3,3,Wing (Spot Up+Transition)
4,Tre Mann,1367,22.0,0.501,0.899,-3.3,-1.1,-4.3,1,Ballhandler (PnR)
5,Aleksej Pokusevski,1233,20.0,0.485,0.808,-2.7,0.1,-2.6,3,Wing (Spot Up+Transition)
6,Aaron Wiggins,1209,15.3,0.556,0.974,-3.4,-0.9,-4.3,3,Wing (Spot Up+Transition)
7,Jeremiah Robinson-Earl,1087,15.4,0.526,0.938,-1.9,-0.3,-2.2,3,Wing (Spot Up+Transition)
8,Kenrich Williams,1072,15.6,0.532,0.928,-1.3,1.1,-0.3,3,Wing (Spot Up+Transition)
9,Isaiah Roby,948,19.0,0.607,1.078,0.4,0.2,0.6,3,Wing (Spot Up+Transition)


### Houston Rockets (#3)

- Point Diff: -9.1 (30th)
- Offense: 108.7 (26th)
- Defense: 117.8 (30th)

In [31]:
display_team_distribution('Houston Rockets')

2021-22 Houston Rockets


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Jalen Green,2138,23.7,0.547,0.974,-0.5,-2.4,-2.9,1,Ballhandler (PnR)
1,Christian Wood,2094,23.3,0.595,1.054,2.1,-0.4,1.6,2,Ballhandler (Iso+PnR)
2,Jae'Sean Tate,2056,19.6,0.567,0.971,-1.3,-0.5,-1.8,3,Wing (Spot Up+Transition)
3,Kevin Porter Jr.,1907,24.1,0.531,0.878,-0.1,-1.6,-1.6,1,Ballhandler (PnR)
4,Garrison Mathews,1712,14.4,0.602,1.150,-1.3,-1.0,-2.3,4,Wing (Hand Off)
5,Eric Gordon,1669,18.6,0.614,1.051,-0.1,-2.2,-2.3,1,Ballhandler (PnR)
6,Kenyon Martin Jr.,1656,16.3,0.603,1.096,0.0,-1.2,-1.2,3,Wing (Spot Up+Transition)
7,Alperen Şengün,1489,22.0,0.552,0.891,-1.6,0.6,-1.0,6,Big (Post Up+Roll)
8,Josh Christopher,1334,21.1,0.534,0.879,-2.2,-1.3,-3.5,3,Wing (Spot Up+Transition)
9,Armoni Brooks,690,17.4,0.490,0.907,-2.6,-1.8,-4.4,4,Wing (Hand Off)


### Sacramento Kings (#4)

- Point Diff: -5.9 (25th)
- Offense: 109.9 (25th)
- Defense: 115.8 (27th)

In [33]:
display_team_distribution('Sacramento Kings')

2021-22 Sacramento Kings


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Harrison Barnes,2587,18.8,0.623,1.125,1.5,-1.3,0.2,3,Wing (Spot Up+Transition)
1,De'Aaron Fox,2083,29.1,0.549,0.971,0.6,-1.2,-0.6,1,Ballhandler (PnR)
2,Davion Mitchell,2076,20.6,0.489,0.865,-2.1,-1.7,-3.8,1,Ballhandler (PnR)
3,Tyrese Haliburton,1757,18.3,0.575,0.982,2.2,0.4,2.6,1,Ballhandler (PnR)
4,Buddy Hield,1574,22.5,0.540,0.972,0.2,-1.5,-1.3,0,Wing (Off Screen)
5,Chimezie Metu,1279,18.5,0.546,0.967,-1.6,0.1,-1.5,3,Wing (Spot Up+Transition)
6,Richaun Holmes,1074,15.8,0.689,1.185,-0.4,-0.1,-0.5,5,Big (Roll+Cut)
7,Damian Jones,1017,16.2,0.701,1.192,-0.2,0.5,0.3,5,Big (Roll+Cut)
8,Maurice Harkless,863,11.2,0.561,0.964,-3.7,0.4,-3.3,3,Wing (Spot Up+Transition)
9,Donte DiVincenzo,665,18.5,0.532,0.902,-0.8,0.1,-0.6,1,Ballhandler (PnR)


### Detroit Pistons (#5)

- Point Diff: -8.0 (26th)
- Offense: 106.2 (28th)
- Defense: 114.2 (23rd)

In [34]:
display_team_distribution('Detroit Pistons')

2021-22 Detroit Pistons


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Saddiq Bey,2704,21.3,0.529,0.984,0.9,-1.2,-0.3,2,Ballhandler (Iso+PnR)
1,Cade Cunningham,2088,27.5,0.504,0.833,-1.1,-0.5,-1.6,1,Ballhandler (PnR)
2,Isaiah Stewart,1816,14.6,0.550,0.952,-2.5,-0.8,-3.3,5,Big (Roll+Cut)
3,Killian Hayes,1647,15.7,0.461,0.750,-4.3,0.8,-3.5,1,Ballhandler (PnR)
4,Cory Joseph,1600,14.3,0.579,0.989,-2.0,-0.1,-2.1,1,Ballhandler (PnR)
5,Jerami Grant,1500,25.7,0.556,1.015,1.4,-0.2,1.2,2,Ballhandler (Iso+PnR)
6,Hamidou Diallo,1269,21.8,0.541,0.989,-1.2,0.7,-0.5,3,Wing (Spot Up+Transition)
7,Frank Jackson,1164,20.9,0.530,0.998,-1.8,-1.7,-3.5,0,Wing (Off Screen)
8,Trey Lyles,990,22.3,0.576,1.043,0.5,-0.4,0.0,3,Wing (Spot Up+Transition)
9,Kelly Olynyk,764,21.5,0.561,0.948,0.5,1.0,1.6,6,Big (Post Up+Roll)


### Indiana Pacers (#6)

- Point Diff: -3.5 (22nd)
- Offense: 113.3 (15th)
- Defense: 116.8 (28th)

In [35]:
display_team_distribution('Indiana Pacers')

2021-22 Indiana Pacers


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Domantas Sabonis,1632,21.7,0.649,1.066,3.6,1.0,4.6,6,Big (Post Up+Roll)
1,Oshae Brissett,1564,17.3,0.530,0.970,-0.5,-1.7,-2.2,3,Wing (Spot Up+Transition)
2,Chris Duarte,1541,21.1,0.537,0.949,-1.2,-1.1,-2.3,3,Wing (Spot Up+Transition)
3,Justin Holiday,1416,15.9,0.568,1.030,-0.8,-0.8,-1.7,4,Wing (Hand Off)
4,Myles Turner,1235,17.1,0.615,1.098,0.1,1.4,1.4,3,Wing (Spot Up+Transition)
5,Caris LeVert,1214,26.8,0.530,0.957,1.0,-1.3,-0.3,1,Ballhandler (PnR)
6,Malcolm Brogdon,1206,24.6,0.557,0.999,2.8,-0.5,2.3,1,Ballhandler (PnR)
7,Torrey Craig,1034,13.9,0.555,0.982,-1.6,-0.6,-2.2,3,Wing (Spot Up+Transition)
8,Duane Washington Jr.,968,22.6,0.521,0.924,-1.9,-2.8,-4.8,3,Wing (Spot Up+Transition)
9,Tyrese Haliburton,938,20.3,0.629,1.018,3.7,-1.0,2.8,1,Ballhandler (PnR)


### Portland Trail Blazers (#7)

- Point Diff: -8.7 (29th)
- Offense: 108.5 (27th)
- Defense: 117.2 (29th)

In [36]:
display_team_distribution('Portland Trail Blazers')

2021-22 Portland Trail Blazers


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Anfernee Simons,1681,24.8,0.583,1.033,1.9,-2.7,-0.7,2,Ballhandler (Iso+PnR)
1,Jusuf Nurkić,1579,23.6,0.586,0.977,0.1,0.2,0.3,6,Big (Post Up+Roll)
2,Robert Covington,1431,12.4,0.519,0.888,-2.9,1.6,-1.3,3,Wing (Spot Up+Transition)
3,Norman Powell,1333,22.5,0.593,1.081,0.6,-1.6,-1.0,3,Wing (Spot Up+Transition)
4,Ben McLemore,1285,21.3,0.567,1.040,-0.7,-2.5,-3.2,0,Wing (Off Screen)
5,CJ McCollum,1267,26.1,0.532,0.969,1.8,-1.7,0.1,2,Ballhandler (Iso+PnR)
6,CJ Elleby,1174,14.8,0.491,0.845,-3.8,-1.5,-5.3,3,Wing (Spot Up+Transition)
7,Nassir Little,1088,15.8,0.575,1.038,-0.9,-0.6,-1.5,3,Wing (Spot Up+Transition)
8,Damian Lillard,1056,29.3,0.550,0.976,4.0,-2.7,1.3,2,Ballhandler (Iso+PnR)
9,Trendon Watford,869,17.6,0.590,1.028,-1.4,-0.4,-1.7,3,Wing (Spot Up+Transition)


### New Orleans Pelicans (#8)

- Point Diff: -2.2 (21st)
- Offense: 111.1 (20th)
- Defense: 113.2 (18th)

In [37]:
display_team_distribution('New Orleans Pelicans')

2021-22 New Orleans Pelicans


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Herbert Jones,2335,13.7,0.573,0.993,-2.0,1.5,-0.5,3,Wing (Spot Up+Transition)
1,Jonas Valančiūnas,2240,23.8,0.617,1.055,2.0,-0.1,1.9,6,Big (Post Up+Roll)
2,Devonte' Graham,2162,19.5,0.514,0.927,0.2,-1.2,-1.0,1,Ballhandler (PnR)
3,Brandon Ingram,1869,29.3,0.554,0.983,2.6,-0.6,2.0,2,Ballhandler (Iso+PnR)
4,Jaxson Hayes,1398,16.4,0.680,1.232,0.6,-0.3,0.2,5,Big (Roll+Cut)
5,Josh Hart,1374,16.7,0.604,1.015,-0.2,0.9,0.7,3,Wing (Spot Up+Transition)
6,Nickeil Alexander-Walker,1317,24.8,0.474,0.841,-1.7,-1.3,-3.0,1,Ballhandler (PnR)
7,Garrett Temple,1098,13.9,0.494,0.868,-2.8,0.3,-2.5,3,Wing (Spot Up+Transition)
8,CJ McCollum,878,29.4,0.580,1.059,5.3,-0.7,4.6,1,Ballhandler (PnR)
9,Trey Murphy III,864,15.7,0.558,1.071,0.2,-1.4,-1.2,3,Wing (Spot Up+Transition)


### San Antonio Spurs (#9)

- Point Diff: -0.3 (20th)
- Offense: 112.6 (18th)
- Defense: 112.9 (16th)

In [38]:
display_team_distribution('San Antonio Spurs')

2021-22 San Antonio Spurs


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Keldon Johnson,2392,21.3,0.574,1.061,0.7,-1.2,-0.5,3,Wing (Spot Up+Transition)
1,Dejounte Murray,2366,27.3,0.533,0.943,3.7,1.7,5.4,1,Ballhandler (PnR)
2,Jakob Poeltl,1970,18.3,0.613,1.071,0.9,0.7,1.6,5,Big (Roll+Cut)
3,Devin Vassell,1937,18.9,0.540,1.010,-0.5,0.0,-0.5,3,Wing (Spot Up+Transition)
4,Lonnie Walker IV,1612,23.6,0.512,0.940,-1.2,-1.3,-2.5,3,Wing (Spot Up+Transition)
5,Derrick White,1486,20.7,0.551,0.974,1.2,0.6,1.8,1,Ballhandler (PnR)
6,Doug McDermott,1223,18.3,0.593,1.095,-0.4,-2.1,-2.5,0,Wing (Off Screen)
7,Tre Jones,1148,15.6,0.545,0.976,-0.9,0.4,-0.5,1,Ballhandler (PnR)
8,Joshua Primo,965,15.4,0.487,0.814,-3.9,-1.1,-5.0,3,Wing (Spot Up+Transition)
9,Keita Bates-Diop,956,14.8,0.578,1.000,-2.1,-0.5,-2.6,3,Wing (Spot Up+Transition)


### Washington Wizards (#10)

- Point Diff: -3.8 (24th)
- Offense: 110.9 (22nd)
- Defense: 114.7 (25th)

In [39]:
display_team_distribution('Washington Wizards')

2021-22 Washington Wizards


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Kentavious Caldwell-Pope,2329,19.0,0.568,1.030,-0.9,-0.6,-1.5,0,Wing (Off Screen)
1,Kyle Kuzma,2204,24.2,0.547,0.941,0.2,-0.4,-0.2,2,Ballhandler (Iso+PnR)
2,Deni Avdija,1984,16.3,0.536,0.942,-2.1,0.4,-1.7,3,Wing (Spot Up+Transition)
3,Corey Kispert,1801,14.6,0.586,1.075,-0.8,-1.5,-2.3,3,Wing (Spot Up+Transition)
4,Daniel Gafford,1444,16.7,0.708,1.258,1.1,0.5,1.6,5,Big (Roll+Cut)
5,Bradley Beal,1439,30.8,0.539,0.936,2.0,-1.2,0.8,2,Ballhandler (Iso+PnR)
6,Raul Neto,1372,18.4,0.538,0.928,-2.5,-0.5,-3.0,1,Ballhandler (PnR)
7,Spencer Dinwiddie,1330,21.0,0.501,0.888,-0.3,-0.7,-0.9,2,Ballhandler (Iso+PnR)
8,Montrezl Harrell,1117,20.8,0.684,1.239,4.3,0.6,4.8,6,Big (Post Up+Roll)
9,Rui Hachimura,943,20.9,0.579,1.075,0.2,-1.7,-1.5,3,Wing (Spot Up+Transition)


### New York Knicks (#11)

- Point Diff: -0.1 (18th)
- Offense: 110.9 (21st)
- Defense: 111.0 (12th)

In [40]:
display_team_distribution('New York Knicks')

2021-22 New York Knicks


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Julius Randle,2544,28.7,0.509,0.868,0.4,0.2,0.5,2,Ballhandler (Iso+PnR)
1,RJ Barrett,2417,27.6,0.511,0.917,-0.3,-1.3,-1.6,1,Ballhandler (PnR)
2,Evan Fournier,2358,20.8,0.554,1.012,-0.4,-0.3,-0.7,3,Wing (Spot Up+Transition)
3,Alec Burks,2318,17.7,0.559,1.017,-0.4,1.2,0.8,1,Ballhandler (PnR)
4,Mitchell Robinson,1848,11.4,0.722,1.260,1.0,1.2,2.2,5,Big (Roll+Cut)
5,Immanuel Quickley,1802,22.2,0.543,0.976,0.2,0.0,0.2,2,Ballhandler (Iso+PnR)
6,Obi Toppin,1230,20.8,0.614,1.110,1.7,0.3,2.0,3,Wing (Spot Up+Transition)
7,Kemba Walker,948,20.8,0.537,0.966,0.7,-0.2,0.4,1,Ballhandler (PnR)
8,Quentin Grimes,786,15.1,0.565,1.015,-1.2,0.9,-0.3,3,Wing (Spot Up+Transition)
9,Derrick Rose,636,22.5,0.541,0.951,2.3,0.0,2.3,1,Ballhandler (PnR)


### Charlotte Hornets (#13, 15)

- Point Diff: +1.2 (15th)
- Offense: 114.8 (6th)
- Defense: 113.6 (20th)

In [41]:
display_team_distribution('Charlotte Hornets')

2021-22 Charlotte Hornets


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Miles Bridges,2837,22.6,0.593,1.069,1.8,-0.3,1.5,2,Ballhandler (Iso+PnR)
1,Terry Rozier,2458,23.2,0.566,1.062,2.4,-0.7,1.7,0,Wing (Off Screen)
2,LaMelo Ball,2422,28.2,0.554,0.944,3.3,0.2,3.4,1,Ballhandler (PnR)
3,Kelly Oubre Jr.,1999,23.0,0.560,1.055,0.2,-1.6,-1.4,3,Wing (Spot Up+Transition)
4,Cody Martin,1866,12.1,0.579,1.026,-1.2,0.7,-0.5,3,Wing (Spot Up+Transition)
5,Mason Plumlee,1793,11.7,0.603,0.954,-1.5,1.5,0.0,5,Big (Roll+Cut)
6,P.J. Washington,1768,15.6,0.588,1.026,-0.2,0.2,-0.1,3,Wing (Spot Up+Transition)
7,Gordon Hayward,1564,20.7,0.570,1.020,0.1,-0.8,-0.7,3,Wing (Spot Up+Transition)
8,Jalen McDaniels,895,15.3,0.595,1.049,-1.9,-0.4,-2.3,3,Wing (Spot Up+Transition)
9,Montrezl Harrell,524,19.0,0.672,1.228,2.6,-0.8,1.8,6,Big (Post Up+Roll)


### Cleveland Cavaliers (#14)

- Point Diff: +1.9 (13th)
- Offense: 111.8 (19th)
- Defense: 110.0 (6th)

In [42]:
display_team_distribution('Cleveland Cavaliers')

2021-22 Cleveland Cavaliers


,player,mp,usg_pct,ts_pct,Overall_PPP,obpm,dbpm,bpm,player_cluster,player_cluster_discription
0,Darius Garland,2430,27.8,0.576,0.974,3.4,-0.8,2.6,1,Ballhandler (PnR)
1,Evan Mobley,2331,20.4,0.549,0.963,-0.9,1.5,0.6,3,Wing (Spot Up+Transition)
2,Isaac Okoro,1981,12.4,0.589,1.050,-2.4,0.7,-1.7,3,Wing (Spot Up+Transition)
3,Lauri Markkanen,1878,19.5,0.582,1.087,0.4,-0.1,0.4,3,Wing (Spot Up+Transition)
4,Jarrett Allen,1809,18.1,0.698,1.212,2.7,1.2,3.9,5,Big (Roll+Cut)
5,Kevin Love,1665,25.1,0.594,1.068,4.3,0.1,4.5,6,Big (Post Up+Roll)
6,Cedi Osman,1462,21.1,0.563,1.014,-0.3,0.5,0.2,3,Wing (Spot Up+Transition)
7,Lamar Stevens,1015,17.1,0.544,0.980,-3.2,0.3,-2.9,3,Wing (Spot Up+Transition)
8,Dean Wade,980,11.1,0.584,1.102,-1.9,1.3,-0.6,3,Wing (Spot Up+Transition)
9,Ricky Rubio,970,24.9,0.488,0.818,-1.0,1.2,0.2,1,Ballhandler (PnR)
